In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import anndata as ad

In [3]:
data_dir = "/large_storage/zhoulab/yanhu/valve_pthlh/data/Valve_PthlhProject_GEO_2025/processed"
results_dir = "/large_storage/zhoulab/yanhu/valve_pthlh/results"

In [36]:
# Get HLHS sample files
hlhs_files = [f for f in os.listdir(data_dir) if f.startswith('HLHS_')]

# Group files by sample
hlhs_samples = {}
for f in hlhs_files:
    # Parse sample info from filename
    parts = f.split('_')
    sample_id = f'{parts[0]}_{parts[1]}_{parts[2]}'
    file_type = parts[3].split('.')[0]
    
    if sample_id not in hlhs_samples:
        hlhs_samples[sample_id] = {}
    hlhs_samples[sample_id][file_type] = os.path.join(data_dir, f)

# Load data for each sample into AnnData objects
hlhs_data = {}
for sample_id, files in hlhs_samples.items():
    # Read the matrix, features and barcodes
    mtx = sc.read_mtx(files['matrix'])
    features = pd.read_csv(files['features'], sep='\t', header=None)
    barcodes = pd.read_csv(files['barcodes'], sep='\t', header=None)
    
    # Create AnnData object
    adata = ad.AnnData(X=mtx.X.T,  # Transpose matrix to match features x cells
                      var=pd.DataFrame(index=features[1]),  # Use gene names as features
                      obs=pd.DataFrame(index=barcodes[0]))  # Use barcodes as observations
    adata.var['gene_names'] = features[1].values
    
    # Add sample ID as annotation
    adata.obs['sample'] = sample_id
    
    hlhs_data[sample_id] = adata

print("Loaded HLHS samples:", list(hlhs_data.keys()))


/home/yanhu/.local/share/mamba/envs/valve_pthlh/lib/python3.11/site-packages/anndata/_core/anndata.py:1794: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/yanhu/.local/share/mamba/envs/valve_pthlh/lib/python3.11/site-packages/anndata/_core/anndata.py:1794: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/yanhu/.local/share/mamba/envs/valve_pthlh/lib/python3.11/site-packages/anndata/_core/anndata.py:1794: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Loaded HLHS samples: ['HLHS_3_PV', 'HLHS_2_MV', 'HLHS_1_AoV', 'HLHS_4_TV']


/home/yanhu/.local/share/mamba/envs/valve_pthlh/lib/python3.11/site-packages/anndata/_core/anndata.py:1794: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [38]:
hlhs_data[sample_id].var

,gene_names
1,
MIR1302-2HG,MIR1302-2HG
FAM138A,FAM138A
OR4F5,OR4F5
AL627309.1,AL627309.1
AL627309.3,AL627309.3
...,...
AC141272.1,AC141272.1
AC023491.2,AC023491.2
AC007325.1,AC007325.1


In [27]:
mtx.shape

(36601, 8293)

In [33]:
adata = hlhs_data[sample_id]

In [35]:
adata.var_names = adata.var['gene_names']

In [31]:
features[1]

0        MIR1302-2HG
1            FAM138A
2              OR4F5
3         AL627309.1
4         AL627309.3
            ...     
36596     AC141272.1
36597     AC023491.2
36598     AC007325.1
36599     AC007325.4
36600     AC007325.2
Name: 1, Length: 36601, dtype: object